In [109]:
#importing all necessary packages and libraries
import pandas as pd
import numpy as np
import requests
import re
import json
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import bs4
import mechanize
import http.cookiejar as cookielib

In [3]:
#importing data
fifa = pd.read_csv('players_22.csv')

/Users/sidgog/Desktop/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (25,108) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
#filtering out playrs below 80 overall rating
fifa = fifa[fifa['overall'] >= 80]
fifa

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,241095,https://sofifa.com/player/241095/nikola-vlasic...,N. Vlašić,Nikola Vlašić,CAM,80,86,33500000.0,66000.0,23,...,57+2,57+2,57+2,61+2,18+2,https://cdn.sofifa.net/players/241/095/22_120.png,https://cdn.sofifa.net/teams/19/60.png,https://cdn.sofifa.net/flags/gb-eng.png,NaN,https://cdn.sofifa.net/flags/hr.png
505,243576,https://sofifa.com/player/243576/pedro-antonio...,Pedro Porro,Pedro Antonio Porro Sauceda,"RWB, RM",80,87,40500000.0,80000.0,21,...,74+2,74+2,74+2,77+2,16+2,https://cdn.sofifa.net/players/243/576/22_120.png,https://cdn.sofifa.net/teams/237/60.png,https://cdn.sofifa.net/flags/pt.png,NaN,https://cdn.sofifa.net/flags/es.png
506,244369,https://sofifa.com/player/244369/viktor-tsygan...,V. Tsygankov,Viktor Tsygankov,RM,80,86,33500000.0,1000.0,23,...,51+2,51+2,51+2,61+2,16+2,https://cdn.sofifa.net/players/244/369/22_120.png,https://cdn.sofifa.net/teams/101047/60.png,https://cdn.sofifa.net/flags/ua.png,https://cdn.sofifa.net/teams/1366/60.png,https://cdn.sofifa.net/flags/ua.png
507,244470,https://sofifa.com/player/244470/vladimir-couf...,V. Coufal,Vladimír Coufal,"RB, RWB",80,80,19000000.0,73000.0,28,...,78+2,78+2,78+2,78+2,18+2,https://cdn.sofifa.net/players/244/470/22_120.png,https://cdn.sofifa.net/teams/19/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1330/60.png,https://cdn.sofifa.net/flags/cz.png


In [5]:
#cleaning data
fifa = fifa.drop([189, 266, 371, 370, 369, 368, 366]).reset_index().drop(columns = ['index'])

In [9]:
fifa = fifa.drop([364]).reset_index().drop(columns = ['index'])

In [13]:
fifa = fifa.drop([434]).reset_index().drop(columns = ['index'])

In [16]:
fifa = fifa.drop([450]).reset_index().drop(columns = ['index'])

In [22]:
fifa = fifa.drop([474, 475, 476, 477]).reset_index().drop(columns = ['index'])

In [28]:
fifa = fifa.drop([264]).reset_index().drop(columns = ['index'])

In [16]:
#functions webscraping likes and dislikes per plyer in dataset
def player_community_likes(df):
    ids = []
    rating = []
    for i in range(len(df)):
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)
        source_code = session.get(df['player_url'][i])
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'html.parser')
        for j in soup.find_all("button"):
            if j['data-action'] == 'like':
                likes = j.find("span").string
                ids.append(df['sofifa_id'][i])
                rating.append(int(likes))
            else:
                break
    return {'sofifa_id':ids, 'likes':rating}

def player_community_dislikes(df):
    rating = []
    ids = []
    for i in range(len(df)):
        session = requests.Session()
        retry = Retry(connect=4, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)
        source_code = session.get(df['player_url'][i])
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'html.parser')
        for j in soup.find_all("button"):
            if j['data-action'] == 'like':
                continue
            elif j['data-action'] == 'dislike':
                dislikes = j.find("span").string
                ids.append(df['sofifa_id'][i])
                rating.append(int(dislikes))
            else:
                break
    return {'sofifa_id':ids, 'dislikes':rating}

In [58]:
#collecting likes per player
likes = player_community_likes(fifa)

NameError: name 'player_community_likes' is not defined

In [6]:
#collecting dislikes per player
dislikes = player_community_dislikes(fifa)

In [8]:
dflikes = pd.DataFrame(likes)
dfdislikes = pd.DataFrame(dislikes)

In [13]:
fifa = fifa.merge(dflikes, how = 'inner', on = 'sofifa_id')
fifa = fifa.merge(dfdislikes, how = 'inner', on = 'sofifa_id')
fifa

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,likes,dislikes
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,1927,645
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png,520,104
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png,1700,517
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png,860,288
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,530,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1962,256790,https://sofifa.com/player/256790/jamal-musiala...,J. Musiala,Jamal Musiala,"CAM, LM",75,88,13000000.0,19000.0,18,...,41+2,51+2,15+2,https://cdn.sofifa.net/players/256/790/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1337/60.png,https://cdn.sofifa.net/flags/de.png,324,36
1963,257536,https://sofifa.com/player/257536/jan-kuchta/22...,J. Kuchta,Jan Kuchta,ST,75,80,8500000.0,950.0,24,...,51+2,53+2,16+2,https://cdn.sofifa.net/players/257/536/22_120.png,https://cdn.sofifa.net/teams/266/60.png,https://cdn.sofifa.net/flags/cz.png,NaN,https://cdn.sofifa.net/flags/cz.png,19,4
1964,259694,https://sofifa.com/player/259694/oscar-minguez...,Mingueza,Óscar Mingueza García,"CB, RB",75,83,12000000.0,76000.0,22,...,75+2,75+2,17+2,https://cdn.sofifa.net/players/259/694/22_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/es.png,83,20
1965,261204,https://sofifa.com/player/261204/myrto-uzuni/2...,M. Uzuni,Myrto Uzuni,"LW, CAM",75,75,6000000.0,1000.0,26,...,44+2,50+2,18+2,https://cdn.sofifa.net/players/261/204/22_120.png,https://cdn.sofifa.net/teams/1874/60.png,https://cdn.sofifa.net/flags/hu.png,NaN,https://cdn.sofifa.net/flags/al.png,4,0


In [21]:
#extracting final data as csv
df_recom = fifa[['player_face_url', 'sofifa_id', 'short_name', 'long_name', 'overall', 'player_positions', 'league_name', 'club_name', 'nationality_name', 'likes', 'dislikes']]
df_recom.to_csv('RatioRecommender.csv')